# Introduction to Metabolic Modeling and Tellurium
Notebook to go through basics of biochemical modeling (ie trying to convert my notes into a coherent document.

First things first: set up the python packages we need.

In [2]:
import tellurium as te # this will be our workhorse package
import math # helpful for some non-Tellurium based calculations
import numpy as np # helpful for some non-Tellurium based calculations
import matplotlib.pyplot as plt # let's us mess around with plots a bit more

# Resources:

We are using [Tellurium](tellurium.analogmachine.org) for do the modeling of biological systems. The [documentation](https://tellurium.readthedocs.io/en/latest/index.html#) is quite useful and full of examples.

There is a useful textbook that you may want to grab a pdf copy of *Systems Biology: Introduction to Pathway Modeling* (on the [analogmachine website](http://books.analogmachine.org/)).

Tellurium makes it very easy to import models from [Biomodels](https://www.ebi.ac.uk/biomodels/) (where you can browse 10k models).

Two resources for finding starting numbers for any number of cell related things: [Bionumbers](https://bionumbers.hms.harvard.edu/search.aspx) or [Cell Biology by the Numbers](book.bionumbers.org)

Use [Brenda](https://www.brenda-enzymes.org/) to get the enzyme kinetic information.



## Thought experiment: What is the concentration of a single molecule in an *E. coli* cell?


Start with a cubic cell with a length of 1$\mu m$. This means that the volume of the cell is 1$\mu m^3$.

For concentrations we need to have volume units of liters not $m^3$. There are 1000 L in 1 $m^3$ and 1 mL in 1 $cm^3$. So we get a cell volume of $1 * 10^{-15}$ L.


In [10]:
cell_volume_um3 = 1; # um^3
print(cell_volume_um3, "um^3")
cell_volume_m3 = cell_volume_um3 * (1/(10**6)**3); # m^3
print(cell_volume_m3,"m^3")
cell_volume_L = cell_volume_m3 * 1000; # L
print(cell_volume_L,"L")

1 um^3
1e-18 m^3
1e-15 L


Now to consider the molecule. We think about stuff in terms of moles and as we know from Avogadro one molecule is approximately $1*10^{-24}$ moles.

So to get the concentration of a single molecule in a cell we need to take the number of moles in a molecule and divide it by the volume of the cell.

In [11]:
molecule = 1*10**(-24); # moles
print(molecule,"moles")
concentration = molecule/cell_volume_L; # M
print(concentration,"M")

1e-24 moles
9.999999999999999e-10 M


So the concentration of 1 molecule in an *E. coli* cell is $1*10^{-9}$ M aka 1 nM so this is the lowest possible concentration of a metabolite/substance of interest in a cell. It is worth remembering that this means that while we are in the nM range we can't have halves.

What about the flip side? What is the maximum concentration of stuff in an *E. coli* cell?

**I don't remember how we did this beyond thinking about the concentrations in media?** The total concentration of stuff in an *E. coli* cell is ~100mM.

<h2> Menten Model of Enzyme Kinetics </h2>

The unit process of biochemistry can be represented graphically as Substrate + Enzyme <=> Enzyme-Substrate (Complex) => Product + Enzyme

This is Michaelis-Menten model (but we will call it the Menten model because she deserves all the credit).

We can break this down into three rate parameters, one for each potential direction of reaction flow.
$k_{+1}$ is the kinetic parameter for the forward direction of the first step ($S+E -> C$).
$k_{-1}$ is the kinetic parameter for the reverse direction of the first step ($C -> S+E$).
And $k_{+2}$ is the kinetic parameter for the forward direction of the second step ($C -> P + E$).

The net rate of the reaction is simply the rate of accumulation of the product.
So we can define the reaction velocity as $V(S) = \frac{dP}{dt}$.
This reaction velocity does not depend on the concentration of product, but is dependent on the concentration of substrate.

Because the full reaction velocity is just the accumulation of product we can easily define it as $V(S) = k_{+2} * C$.

Menten assumed that the concentration of the substrate was much much greater than the concentration of the enzyme.
This means that as soon as enzyme is freed up by $k_{+2}$ it can be incorporated into a new complex, so the complex is kept at steady concentrations: $\frac{dC}{dt} = 0$

Moving this forward we get a pair of rate laws that reflect these transformations:

1. The consumption of the enzyme-substrate complex is equal to the production of the enzyme-substrate complex: $(k_{+2}+k_{-1}) C = k_{+1} S * E$

   These rate constants have different units. Consumption has units per time. Production has units of per time * something

2. The total enzyme present is equal to the free enzyme times the amount bound up in a complex: $E_{total} = E + C$

    Which we can rearrange to: $E = E_{total} - C$

We can combine these two rate laws (and rearrange a little) to get: $(k_{+2}+k_{-1}) C + k_{+1} S * C = k_{+1} E_{total} * S$

A little more rearranging to look at the concentration of complex:

$$C = \frac{k_{+1} E_{total} * S} {(k_{+2}+k_{-1}) + k_{+1} * S} = \frac{E_{total} * S}{(k_{+2} + \frac{k_{-1}}{k_{+1}}) + S}$$

Now we can consider two endmember scenarios:

**Endmember 1:** If the concentration of the substrate is much much bigger than the kinetic ratio $\frac{k_{+2}+k_{-1}}{k_{+1}}$ then $V = k_{+2} * E_{total}$ which defines $V_{max} = k_{cat} * E_{total}$

**Endmember 2:** if the concentration of the substrate is equal to the kinetic ratio (see above) then $V = \frac{k_{+2} * E_{total} * S}{2 * S} = \frac{k_{+2} * E_{total}}{2} = \frac{V_{max}}{2}$